In [288]:
import graph_tool as gt
import networkx as nx
import numpy as np
from graph_tool import collection
from graph_helpers import load_graph_by_name
from graph_tool.spectral import adjacency, laplacian
from graph_tool.generation import complete_graph, random_graph, random_rewire
from scipy.sparse.linalg import eigs
from scipy.sparse.csgraph import laplacian as scipy_laplacian
from scipy.sparse import dia_matrix, diags

In [381]:
eigs?

In [399]:
def get_eig_vals(name, k=10):
    if name == 'complete':
        g = complete_graph(15)
    elif name == 'random':
        g = random_graph(100, lambda: 10, directed=False)
    elif name == 'karate':
        g = collection.data['karate']
    elif name == 'dolphins':
        g = collection.data['dolphins']
    else:
        g = load_graph_by_name(name)
    if False:
        adj = adjacency(g)
        w_adj, v= eigs(adj, k=k, which='LR')
        lap = laplacian(g)
        w_lap, v= eigs(lap, k=2, which='LR')
    else:
        A = adjacency(g)
        degs  = A.sum(axis=1)
        degs = np.array(degs.T.tolist()[0])
        D_inv = diags(1/degs, 0)

        # A_norm = D_inv * A 
        A_norm = A * D_inv
        L = scipy_laplacian(A_norm)
        w_adj, v= eigs(A_norm, k=k, which='LR')
        w_lap, v = eigs(L, k=2, which='LR')
    return {'g': g, 'adj': w_adj, 'lap': w_lap}

In [400]:
graphs = ['lattice-1024', 'infectious', 'grqc', 'p2p']

In [401]:
g = load_graph_by_name('lattice-1024')
A = adjacency(g)
degs  = A.sum(axis=1)
degs = np.array(degs.T.tolist()[0])
D_inv = diags(1/degs, 0)

A_norm = D_inv * A
L = scipy_laplacian(A_norm)


load graph from data/lattice-1024/graph.gt


In [402]:
eig_vals = {}
for g in graphs:
    eig_vals[g] = get_eig_vals(g)

load graph from data/lattice-1024/graph.gt
load graph from data/infectious/graph.gt
load graph from data/grqc/graph.gt
load graph from data/p2p/graph.gt


In [367]:
def non_randomness(vals, k=2):
    return (vals[:k].sum()).real

def mean_var_ER(g, k=2):
    n = g.num_vertices()
    m = g.num_edges()
    p = 2 * k * m / n / (n - k)   
    mean = (n - 2*k) * p + k
    v = 2 * k * p * (1 - p)
    return mean, np.sqrt(v)

def relative_non_randomess(g, eigs, k):
    m, v = mean_var_ER(g, k)
    nr = non_randomness(eigs, k)
    return (nr - m) / v

In [403]:
def spectral_gap(vals):
    return (vals[0] - vals[1]).real

In [404]:
for k, v in eig_vals.items():
    print(k, ', '.join(map(lambda s: '{:.4f}'.format(s), np.real(v['adj'])[:5])))

p2p 1.0000, 0.9596, 0.9190, 0.9037, 0.9000
infectious 1.0000, 0.9820, 0.9564, 0.9257, 0.9011
grqc 1.0000, 0.9981, 0.9979, 0.9963, 0.9898
lattice-1024 1.0000, 0.9949, 0.9975, 0.9975, 0.9795


In [406]:
for k, v in eig_vals.items():
    print('{}: {}'.format(k, spectral_gap(v['adj'])))

p2p: 0.04037015147737566
infectious: 0.017955918982718533
grqc: 0.0018672256377949559
lattice-1024: 0.005130676608106177


In [212]:
{k: relative_non_randomess(v['g'], v['adj'], 2) for k, v in eig_vals.items()}

{'dolphins': 1.612029197614354,
 'grqc': 618.2297090129925,
 'infectious': 28.7727592585828,
 'karate': 1.2159525100083168,
 'lattice-1024': -56.10816632938739,
 'p2p': -80.2691910803632}

In [220]:
{k: relative_non_randomess(v['g'], v['adj'], 3) for k, v in eig_vals.items()}

{'dolphins': 0.330621919266615,
 'grqc': 571.5544389347444,
 'infectious': 24.529948692170358,
 'karate': -6.7516753103493405,
 'lattice-1024': -41.03098729443478,
 'p2p': -217.85710389867603}

In [218]:
{k: relative_non_randomess(v['g'], v['adj'], 6) for k, v in eig_vals.items()}

{'dolphins': -6.813283496909035,
 'grqc': 416.7957200918325,
 'infectious': 14.738173333566154,
 'karate': -54.024005393298744,
 'lattice-1024': -41.02493614236949,
 'p2p': -142.83128897341496}

In [194]:
rand_g = random_graph(100, lambda: 10, directed=False)
mean_var_ER(rand_g, 2)

(21.591836734693878, 0.8060578635972041)

In [207]:
def regular_measure(g, k):
    n = g.num_vertices()
    m = g.num_edges()
    p = 2 * k * m / n / (n - k)   
    return - k / (2 * k * p * (1 - p))

In [208]:
regular_measure(eig_vals['p2p']['g'], k=2)

-239.14662038840012